# Fillter

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:

filepath = '../../data/raw/df.xlsx'
df = pd.read_excel(filepath)

# 5年ごとにする
# df = df[df['years'] % 5 == 0]

# 遠すぎる島を除外
df = df[df['distance_m'] <= 1991]

# 人口対数値を追加
df['log_pop'] = np.log(df['population'])

# 経過年数を追加
df['time_since_treatment'] = np.where(df['treatment_group'] == 1, df['year'] - df['bridge_year'], 0)
df['pseudo_time_since_treatment'] = np.where(df['treatment_group'] == 1, df['pseudo_year'] - df['pseudo_bridge_year'], 0)

# 疑似介入後ダミーを追加
df['pseudo_after_treatment'] = np.where(df['pseudo_time_since_treatment'] > 0, 1, 0)

# df.dropna(subset='population_change_rate', inplace=True)

# 同じisland_id内でafter_treatment == 0がないものを除外
# already_treated = df.groupby('island_ids').filter(lambda x: (x['after_treated'] == 0).sum() == 0)
# df = df[~df['island_ids'].isin(already_treated['island_ids'])]

# 観測期間が短い島を除外
# missing_1975_islands = df[df['years'] == 1975]['island_ids'].unique()
# all_islands = df['island_ids'].unique()
# islands_without_1975 = [island for island in all_islands if island not in missing_1975_islands]
# print(islands_without_1975)
# df = df[~df['island_ids'].isin(islands_without_1975)]

In [4]:
# 地域名，島名，県名を削除
df.drop(columns=['region_names', 'prefecture_names', 'island_names'], inplace=True)
df = df[['island_ids'] + [col for col in df.columns if col != 'island_ids']]

In [5]:
df

,island_ids,year,pseudo_year,region_codes,prefecture_codes,population,treatment_group,treatment_group_mainland,bridge_year,pseudo_bridge_year,...,area_km2,distance_m,elementary_school,junior_high_school,high_school,hospital,log_pop,time_since_treatment,pseudo_time_since_treatment,pseudo_after_treatment
119,67.0,1970.0,1970.0,32202,32.0,NaN,1.0,1.0,1999.0,1999.0,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,-29.0,-29.0,0
120,67.0,1975.0,1975.0,32202,32.0,NaN,1.0,1.0,1999.0,1999.0,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,-24.0,-24.0,0
121,67.0,1980.0,1980.0,32202,32.0,NaN,1.0,1.0,1999.0,1999.0,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,-19.0,-19.0,0
122,67.0,1985.0,1985.0,32202,32.0,NaN,1.0,1.0,1999.0,1999.0,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,-14.0,-14.0,0
123,67.0,1990.0,1990.0,32202,32.0,NaN,1.0,1.0,1999.0,1999.0,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,-9.0,-9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,226.0,2000.0,2000.0,39424,39.0,398.0,1.0,1.0,1957.0,1957.0,...,0.56,110.0,NaN,NaN,NaN,NaN,5.986452,43.0,43.0,1
3475,226.0,2005.0,2005.0,39424,39.0,365.0,1.0,1.0,1957.0,1957.0,...,0.56,110.0,NaN,NaN,NaN,NaN,5.899897,48.0,48.0,1
3476,226.0,2010.0,2010.0,39424,39.0,343.0,1.0,1.0,1957.0,1957.0,...,0.56,110.0,NaN,NaN,NaN,NaN,5.837730,53.0,53.0,1
3477,226.0,2015.0,2015.0,39424,39.0,284.0,1.0,1.0,1957.0,1957.0,...,0.56,110.0,NaN,NaN,NaN,NaN,5.648974,58.0,58.0,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1857 entries, 119 to 3478
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   island_ids                1857 non-null   float64
 1   year                      1857 non-null   float64
 2   region_codes              1857 non-null   object 
 3   prefecture_codes          1857 non-null   float64
 4   population                1694 non-null   float64
 5   treatment_group           1857 non-null   float64
 6   treatment_group_mainland  1857 non-null   float64
 7   bridge_year               1857 non-null   float64
 8   after_treatment           1857 non-null   float64
 9   connect_mainland_year     1857 non-null   float64
 10  after_treatment_mainland  1857 non-null   float64
 11  income                    1401 non-null   float64
 12  area_km2                  1857 non-null   float64
 13  distance_m                1857 non-null   float64
 14  elementary_

In [6]:
df_filtered = df
export_filepath = '../../data/processed/df_filtered.xlsx'
df_filtered.to_excel(export_filepath, index=False)